In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [20]:
# Import files to use in preprocessing and machine learning
from implementations import *
from proj1_helpers import *
from preprocess import *
from cross_validation import *

## Load the training data into feature matrix, class labels, and event ids:

In [4]:
# Download train data and supply path here 
DATA_TRAIN_PATH = 'data_aicrowd/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [5]:
# Check the array shape of y, tX, and ids
print(y.shape)
print(tX.shape)
print(ids.shape)

(250000,)
(250000, 30)
(250000,)


# Initial Data Analysis

In observing the original training data, we found out that there exists missing data all over tX. The missing data are represented as value -999. Considering that these columns are critical in model training, we cannot simply delete these rows with -999 values. Therefore, we need to process the original training set before model training.

Firstly, we check the columns of tX to obtain an overview of missing data:

In [6]:
# Check whether the missing values are associated with the classification result
for col in range(tX.shape[1]):
    tX_T = np.transpose(tX)
    
    null = (tX_T[col] == -999)
    null_s = np.logical_and(y >= 0, null)
    null_b = np.logical_and(y < 0, null)
    
    tX_null = tX[null]
    tX_null_s = tX[null_s]
    tX_null_b = tX[null_b]
    
    if (tX_null.shape[0] > 0):
        # Print the percentage of column 'col' having a -999 (missing) value
        print('Column', col, 'has {}% percentage of missing values'.format(tX_null.shape[0] * 100 / tX.shape[0]))

        # Print the conditional probability of P(y = 1|x having -999)
        print('P(y = 1|x having -999) = {:.3f}%'.format(tX_null_s.shape[0] * 100 / tX_null.shape[0]))
        
        # Print the conditional probability of P(y = -1|x having -999)
        print('P(y = -1|x having -999) = {:.3f}% \n'.format(tX_null_b.shape[0] * 100 / tX_null.shape[0]))

Column 0 has 15.2456% percentage of missing values
P(y = 1|x having -999) = 7.438%
P(y = -1|x having -999) = 92.562% 

Column 4 has 70.9828% percentage of missing values
P(y = 1|x having -999) = 29.980%
P(y = -1|x having -999) = 70.020% 

Column 5 has 70.9828% percentage of missing values
P(y = 1|x having -999) = 29.980%
P(y = -1|x having -999) = 70.020% 

Column 6 has 70.9828% percentage of missing values
P(y = 1|x having -999) = 29.980%
P(y = -1|x having -999) = 70.020% 

Column 12 has 70.9828% percentage of missing values
P(y = 1|x having -999) = 29.980%
P(y = -1|x having -999) = 70.020% 

Column 23 has 39.9652% percentage of missing values
P(y = 1|x having -999) = 25.514%
P(y = -1|x having -999) = 74.486% 

Column 24 has 39.9652% percentage of missing values
P(y = 1|x having -999) = 25.514%
P(y = -1|x having -999) = 74.486% 

Column 25 has 39.9652% percentage of missing values
P(y = 1|x having -999) = 25.514%
P(y = -1|x having -999) = 74.486% 

Column 26 has 70.9828% percentage of 

We can see that 11 columns contains at least one -999 (missing value). Now we check whether some of the missing values are dependent on the column named 'PRI_jet_num' (column No. 23), since 'PRI_jet_num' has a discrete value range {0, 1, 2, 3} and our observation on the beginning data rows showed a dependency of some missing values to the value of 'PRI_jet_num' column.

In [7]:
PRI_jet_range = [i for i in range(0, 4)]
PRI_jet_sum = []
PRI_jet_null = []

for value in PRI_jet_range:
    tX_PRI = tX[tX[:, 22] == value]
    
    # Append values of row numbers for different PRI_jet_num, finally sum up to see whether it equals to the length of tX
    PRI_jet_sum.append(len(tX_PRI))
    
    # Count the number of missing columns corresponding to different PRI_jet_num values
    PRI_jet_keys = []
    for i in range (len(tX_PRI)):
        tX_null_cols = np.count_nonzero(tX_PRI[i] == -999, axis = 0)
        PRI_jet_keys.append(tX_null_cols)
    
    PRI_jet_null.append(list(set(PRI_jet_keys)))

    
print("Sum of rows for different PRI_jet_num: {} \n".format(sum(PRI_jet_sum)))

for i in range(4):
    print("PRI_jet_num =", PRI_jet_range[i], "No. of columns having -999 (a missing value):{}".format(PRI_jet_null[i]))

Sum of rows for different PRI_jet_num: 250000 

PRI_jet_num = 0 No. of columns having -999 (a missing value):[10, 11]
PRI_jet_num = 1 No. of columns having -999 (a missing value):[8, 7]
PRI_jet_num = 2 No. of columns having -999 (a missing value):[0, 1]
PRI_jet_num = 3 No. of columns having -999 (a missing value):[0, 1]


The above analysis showed that one column with -999 (missing value) is independent of the column 'PRI_jet_num', we check the original training set and we can easily find out that the first tX column 'DER_mass_MMC' is independent of 'PRI_jet_num'. 

# Data Preprocessing

Based on the data analysis above, we conduct the following method to pre-process the training data. 

In [15]:
# Split the database based on 'PRI_jet_num' (0, 1, 2&3)
tX_jet_0, tX_jet_1, tX_jet_23, y_jet_0, y_jet_1, y_jet_23, r_ids = split_reformat_data(tX, y, ids)

print(tX_jet_0.shape)
print(tX_jet_1.shape)
print(tX_jet_23.shape)

(99913, 18)
(77544, 22)
(72543, 29)


In [17]:
# Replacing the missing values of first feature column with median
replace_missing_value(tX_jet_0, 0, 'median')
replace_missing_value(tX_jet_1, 0, 'median')
replace_missing_value(tX_jet_23, 0, 'median')

In [19]:
# Standardize the splitted training data
tx_0 = standardize(tX_jet_0)
tx_1 = standardize(tX_jet_1)
tx_23 = standardize(tX_jet_23)

# Model Training

## Least Squares

## Ridge Regression

In [ ]:
# Set H-parameters
K_FOLD = 10
DEGREE = np.arange(2, 11)
SEED = 5
LAMBDA = np.logspace(-8, -6, 20)

In [ ]:
def find_optimal(degrees, k_fold, lambdas, seed=1):
    # Split the data into k-fold
    k_indices = build_k_indices(y, k_fold, seed)
    
    # Set lists for collecting best lambda & rmse for each degree
    best_lambda = []
    best_rmse = []
    
    for degree in degrees:
        rmse_val = []
        
        for lambda_ in lambdas:
            rmse_val_lambda_ = []
            
            for k in range(k_fold):
                _, loss_val, w = cross_validation(y, x, k_indices, k, lambda_, degree)
                rmse_val_lambda_.append(loss_val)

            rmse_val.append(np.mean(rmse_val_lambda_))
        
        index_opt_lambda = np.argmin(rmse_val)
        best_lambda.append(lambdas[index_opt_lambda])
        best_rmse.append(rmse_val[index_opt_lambda])
    
    opt_degree = degrees[np.argmin(best_rmse)]
    opt_lambda = best_lambda[np.argmin(best_rmse)]
    
    return opt_degree, opt_lambda

In [ ]:
# Find the 

## Logistic Regression

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = 'data_aicrowd/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = 'data_aicrowd/pred.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)